# 90803 Data Cleaning and Question Definition
# Merge Crime Data (city level) with ACS data (city level)
#### The city level of the crime data and ACS data don't match up exactly so we will have to try our best with string matching on the city names

**Team 14**

Colton Lapp & Chi-Shiun Tsai

In [1]:
#import modules 
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import geopandas as gpd
from census import Census
from us import states
import numpy as np
import os
import seaborn as sns
import us
import re

# Read in Crime Data
This should be on your local machine from step 4

In [2]:
#Read in DFs
base_dir = '/'.join( os.getcwd().split('/')[0:-1] )
base_dir
crime = pd.read_csv(base_dir+ '/data/data_cleaned/crime2020.csv')
crime.head()

,City,Population,TotalOffenses,State
0,Alta Vista,422,0,Kansas
1,Andover,13592,553,Kansas
2,Anthony,2051,115,Kansas
3,Arkansas City,11589,1272,Kansas
4,Arma,1413,87,Kansas


## Drop the word "township", "village", "city", etc from city names in crime data
These words are messing up our merges because they are inconsistent across datsets so we will drop them both

In [3]:
crime['City'] = crime['City'].apply(  lambda x: re.sub(" Township", "", x,flags=re.I) )
crime['City'] = crime['City'].apply(  lambda x: re.sub(" Town", "", x,flags=re.I) )
crime['City'] = crime['City'].apply(  lambda x: re.sub(" City", "", x,flags=re.I) )
crime['City'] = crime['City'].apply(  lambda x: re.sub(" Village", "", x,flags=re.I) )
crime['City'] = crime['City'].apply(  lambda x: re.sub("Metropolitan ", "", x,flags=re.I) )
crime['City'] = crime['City'].apply(  lambda x: re.sub(" Police Department", "", x,flags=re.I) )
crime['City'] = crime['City'].apply(  lambda x: re.sub("-Mecklenburg", "", x,flags=re.I) )
crime['City'] = crime['City'].apply(  lambda x: re.sub("-Clarke County", "", x,flags=re.I) )



crime['City'] = crime['City'].apply(  lambda x: x.strip() )
crime.head()

,City,Population,TotalOffenses,State
0,Alta Vista,422,0,Kansas
1,Andover,13592,553,Kansas
2,Anthony,2051,115,Kansas
3,Arkansas,11589,1272,Kansas
4,Arma,1413,87,Kansas


## Create "full name" column combining city and state

In [4]:
crime['full_name'] = crime['City'] + ', ' + crime['State']

# Read in Places shape file to merge onto crime data
#### This should be on your machine from step 1

In [5]:
places_shp = gpd.read_file( base_dir + '/data/shape_files/CensusPlaces')
places_shp.head(1)

,STATEFP,PLACEFP,PLACENS,GEOID,NAME,NAMELSAD,LSAD,CLASSFP,PCICBSA,PCINECTA,MTFCC,FUNCSTAT,ALAND,AWATER,INTPTLAT,INTPTLON,geometry
0,01,05932,02405250,0105932,Berry,Berry town,43,C1,N,N,G4110,A,27884733,15029,+33.6667018,-087.6093110,"POLYGON ((-87.63910 33.66662, -87.63794 33.666..."


### Remove "city", "Village", "town", etc from NAME variable in Census Shapefiles to make merges better

In [6]:
places_shp['NAME'] = places_shp['NAME'].apply(  lambda x: re.sub(" Township", "", x,flags=re.I) )
places_shp['NAME'] = places_shp['NAME'].apply(  lambda x: re.sub(" Town", "", x,flags=re.I) )
places_shp['NAME'] = places_shp['NAME'].apply(  lambda x: re.sub(" City", "", x,flags=re.I) )
places_shp['NAME'] = places_shp['NAME'].apply(  lambda x: re.sub(" Township", "", x,flags=re.I) )
places_shp['NAME'] = places_shp['NAME'].apply(  lambda x: re.sub(" Village", "", x,flags=re.I) )
places_shp['NAME'] = places_shp['NAME'].apply(  lambda x: re.sub(" CDP", "", x,flags=re.I) )
places_shp['NAME'] = places_shp['NAME'].apply(  lambda x: re.sub("\(balance\)", "", x,flags=re.I) )
places_shp['NAME'] = places_shp['NAME'].apply(  lambda x: re.sub("-Davidson metropolitan government", "", x,flags=re.I) )
places_shp['NAME'] = places_shp['NAME'].apply(  lambda x: re.sub("-Fayette", "", x,flags=re.I) )
places_shp['NAME'] = places_shp['NAME'].apply(  lambda x: re.sub("-Clarke County unified government", "", x,flags=re.I) )

places_shp['NAME'] = places_shp['NAME'].apply(  lambda x: x.strip() )

In [7]:
#Add state name to shape file
places_shp['State Name'] = places_shp['STATEFP'].apply( lambda x: str(us.states.lookup(x)) )
places_shp.head(1)

#Create "Full name" by combining state and city name
places_shp['Full_name_1'] =  places_shp['NAME'] + ', ' + places_shp['State Name']
places_shp['Full_name_2'] =  places_shp['NAMELSAD'] + ', ' + places_shp['State Name']
places_shp.head(3)

,STATEFP,PLACEFP,PLACENS,GEOID,NAME,NAMELSAD,LSAD,CLASSFP,PCICBSA,PCINECTA,MTFCC,FUNCSTAT,ALAND,AWATER,INTPTLAT,INTPTLON,geometry,State Name,Full_name_1,Full_name_2
0,01,05932,02405250,0105932,Berry,Berry town,43,C1,N,N,G4110,A,27884733,15029,+33.6667018,-087.6093110,"POLYGON ((-87.63910 33.66662, -87.63794 33.666...",Alabama,"Berry, Alabama","Berry town, Alabama"
1,01,25840,02403599,0125840,Fayette,Fayette city,25,C1,N,N,G4110,A,22143483,212108,+33.6942153,-087.8311690,"POLYGON ((-87.85507 33.70779, -87.85510 33.710...",Alabama,"Fayette, Alabama","Fayette city, Alabama"
2,01,32536,02406632,0132536,Gu-Win,Gu-Win town,43,C1,N,N,G4110,A,5031111,0,+33.9443303,-087.8703764,"POLYGON ((-87.88578 33.95916, -87.88577 33.959...",Alabama,"Gu-Win, Alabama","Gu-Win town, Alabama"


# Join crime to city shapefile by name (imperfect strategy)

In [8]:
merged = places_shp.merge(crime, left_on='Full_name_1', right_on='full_name', how='outer')


## Show Crime Observations that didn't get matched

In [9]:
pd.set_option('display.max_rows', 100)

missing = merged[ merged['STATEFP'].isnull() ].copy()

#See how many were not merged
print( "Total Crime observations that were not merged: ", missing.shape[0], "out of", crime.shape[0])
missing.sort_values('Population', ascending=False)[['City', 'State', 'Population']].head(20)

Total Crime observations that were not merged:  458 out of 4674


,City,State,Population
29884,Canton,Michigan,94128.0
29950,Clarkstown,New York,80462.0
29934,Waterford,Michigan,72735.0
29935,West Bloomfield,Michigan,65711.0
30105,West Chester,Ohio,62685.0
30143,Fairfield,Connecticut,62311.0
30148,Hamden,Connecticut,60440.0
30078,Colerain,Ohio,59307.0
29921,Redford,Michigan,46510.0
29957,Webster and,New York,45449.0


# Do Inner join of crime and city data
This is the best we can do with improperly matching cities. We did our best to manually look through the data and correct mistakes

In [10]:
#Do Inner Join and ignore missing matches
merged = places_shp.merge(crime, left_on='Full_name_1', right_on='full_name', how='inner')
merged = merged[['STATEFP', 'PLACEFP', 'GEOID', 'NAMELSAD', 'State Name', 'City', 'TotalOffenses']]

# Save Dataset

In [11]:
merged.to_csv( base_dir + '/data/data_cleaned/crime_cities.csv', index=False)

# References
1. https://www.statology.org/pandas-merge-on-different-column-names/
2. https://stackoverflow.com/questions/8813265/why-doesnt-ignorecase-flag-re-i-work-in-re-sub
3. https://stackoverflow.com/questions/38773379/simplest-python-equivalent-to-rs-gsub
4. https://www.geeksforgeeks.org/select-rows-that-contain-specific-text-using-pandas/#
